In [34]:
%pip install pandas numpy yfinance edgartools openai


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Import required libraries
import numpy as np
import yfinance as yf
from edgar import set_identity, Company  # Updated imports per edgartools documentation
import re  # Added import for regular expressions
import logging
from openai import OpenAI
import json

# Set up basic logging for debugging and feedback
logging.basicConfig(level=logging.INFO)

# Set your identity for SEC EDGAR access (required by edgartools)
set_identity("morksen@example.com")  # Replace with your actual name and email


INFO:edgar.core:Identity of the Edgar REST client set to [morksen@example.com]


In [ ]:
XAI_API_KEY = 'INSERT YOUR OWN'

In [37]:
class QualitativeAgent:
    """Handles qualitative analysis using SEC filings and the Grok API."""
    def __init__(self, ticker, api_key):
        self.ticker = ticker
        self.api_key = api_key
    
    def get_business_model_info(self):
        """Fetches the latest 10-K and analyzes it with Grok for detailed qualitative insights."""
        try:
            company = Company(self.ticker)
            filings = company.get_filings(form="10-K")
            latest_filing = filings.latest(1)
            if not latest_filing:
                raise ValueError(f"No 10-K filing found for {self.ticker}")
            
            filing_text = latest_filing.text()
            business_text = self._extract_section(filing_text, "Item 1. Business")
            if not business_text:
                raise ValueError("Business section not found in the filing")
            
            grok_response = self._analyze_with_grok(business_text)
            analysis = self._parse_grok_response(grok_response)
            qualitative_info = {
                "business_model": analysis.get("business_model", "unknown"),
                "competitive_position": analysis.get("competitive_position", "N/A"),
                "growth_drivers": analysis.get("growth_drivers", "N/A"),
                "risks": analysis.get("risks", "N/A"),
                "narrative": analysis.get("narrative", "")
            }
            logging.info(f"Qualitative data extracted for {self.ticker}")
            return qualitative_info
        
        except Exception as e:
            logging.error(f"Error in qualitative data retrieval: {e}")
            return {"business_model": "unknown", "competitive_position": "N/A", "growth_drivers": "N/A", "risks": "N/A", "narrative": ""}
    
    def _extract_section(self, filing_text, section_name):
        """Extracts a specific section from the filing text using regex."""
        pattern = rf"{section_name}\.?(.*?)(Item \d+\.|$)"
        match = re.search(pattern, filing_text, re.DOTALL | re.IGNORECASE)
        return match.group(1).strip() if match else ""
    
    def _analyze_with_grok(self, text):
        """Send the extracted text to the Grok API for qualitative analysis."""
        client = OpenAI(
            api_key=self.api_key,
            base_url="https://api.x.ai/v1",
        )

        completion = client.chat.completions.create(
            model="grok-2-latest",
            messages=[
                {
                    "role": "system",
                    "content": "You are an investor doing in-depth research on businesses."
                },
                {
                    "role": "user",
                    "content": (
                        "Analyze the business based on the provided text. Include the following in your analysis:\n"
                        "1. Business model\n"
                        "2. Competitive position and market dominance\n"
                        "3. Key growth drivers\n"
                        "4. Potential risks and challenges\n"
                        "5. A brief narrative summary\n\n"
                        "Return the analysis in this format:\n"
                        "qualitative_info = {\n"
                        "    \"business_model\": \"value\",\n"
                        "    \"competitive_position\": \"value\",\n"
                        "    \"growth_drivers\": \"value\",\n"
                        "    \"risks\": \"value\",\n"
                        "    \"narrative\": \"value\"\n"
                        "}"
                    )
                }
            ]
        )
        return completion.choices[0].message.content
    
    def _parse_grok_response(self, response):
        """Parse the Grok API response into a dictionary."""
        try:
            match = re.search(r"qualitative_info\s*=\s*\{[^}]*\}", response)
            if match:
                dict_str = match.group(0).replace("qualitative_info =", "").strip()
                return json.loads(dict_str)
            else:
                logging.warning("No valid qualitative_info format found in Grok response")
                return {}
        except (json.JSONDecodeError, AttributeError) as e:
            logging.error(f"Failed to parse Grok response: {e}")
            return {}

In [38]:
class QuantitativeAgent:
    """Handles quantitative analysis using Yahoo Finance data."""
    def __init__(self, ticker):
        self.ticker = ticker
    
    def get_financial_data(self):
        """Fetches financial metrics from Yahoo Finance."""
        try:
            stock = yf.Ticker(self.ticker)
            income_statement = stock.quarterly_income_stmt.T
            cash_flow = stock.quarterly_cashflow.T
            
            revenue_growth = self._compute_qoq_revenue_growth(income_statement)
            profit_growth = self._compute_qoq_profit_growth(income_statement)
            cash_flow_stability = self._assess_cash_flow_stability(cash_flow)
            
            quantitative_info = {
                "revenue_growth": revenue_growth,
                "profit_growth": profit_growth,
                "cash_flow_stability": cash_flow_stability,
                "EPS": stock.info.get("trailingEps", None),
                "free_cash_flow": cash_flow.get("Free Cash Flow", np.nan).iloc[0] if not cash_flow.empty else np.nan
            }
            logging.info(f"Quantitative data retrieved for {self.ticker}")
            return quantitative_info
        except Exception as e:
            logging.error(f"Error in quantitative data retrieval: {e}")
            return {}
    
    def _compute_qoq_revenue_growth(self, income_statement):
        """Computes quarter-over-quarter revenue growth from quarterly financials."""
        try:
            revenues = income_statement["Total Revenue"].dropna()
            if len(revenues) >= 2:
                latest_revenue = revenues.iloc[0]
                previous_revenue = revenues.iloc[1]
                return (latest_revenue - previous_revenue) / previous_revenue
            return 0.0
        except Exception as e:
            logging.error(f"Error computing QoQ revenue growth: {e}")
            return 0.0
    
    def _compute_qoq_profit_growth(self, income_statement):
        """Computes quarter-over-quarter profit growth from quarterly financials."""
        try:
            profits = income_statement["Net Income"].dropna()
            if len(profits) >= 2:
                latest_profit = profits.iloc[0]
                previous_profit = profits.iloc[1]
                return (latest_profit - previous_profit) / previous_profit
            return 0.0
        except Exception as e:
            logging.error(f"Error computing QoQ profit growth: {e}")
            return 0.0
    
    def _assess_cash_flow_stability(self, cash_flow):
        """Assesses cash flow stability based on free cash flow trends."""
        try:
            fcf = cash_flow["Free Cash Flow"].dropna()
            if len(fcf) >= 4:
                if all(fcf.iloc[:4] > 0):
                    return "Stable"
            return "Unstable"
        except Exception as e:
            logging.error(f"Error assessing cash flow stability: {e}")
            return "Unstable"

In [39]:
class ScoringAgent:
    """Computes a combined score from qualitative and quantitative data."""
    def compute_score(self, qualitative_data, quantitative_data):
        """Calculates a final score based on weighted averages of multiple factors."""
        # Extract quantitative metrics
        revenue_growth = quantitative_data.get("revenue_growth", 0)
        profit_growth = quantitative_data.get("profit_growth", 0)
        cash_flow_stability = 1 if quantitative_data.get("cash_flow_stability", "Unstable") == "Stable" else 0
        
        # Normalize metrics (assume growth is in decimal form, e.g., 2.17 for 217%)
        revenue_growth_score = min(max(revenue_growth * 100, 0), 100)
        profit_growth_score = min(max(profit_growth * 100, 0), 100)
        
        # Assign weights
        weight_revenue = 0.3
        weight_profit = 0.3
        weight_cash_flow = 0.2
        weight_qualitative = 0.2
        
        # Qualitative score based on business model and narrative
        qualitative_score = 0.5 if qualitative_data.get("business_model", "").lower() == "subscription" else 0
        if "growth" in qualitative_data.get("narrative", "").lower():
            qualitative_score += 0.5
        
        # Compute weighted scores
        score = (
            (revenue_growth_score * weight_revenue) +
            (profit_growth_score * weight_profit) +
            (cash_flow_stability * 100 * weight_cash_flow) +
            (qualitative_score * 100 * weight_qualitative)
        )
        
        return min(score, 100)  # Cap score at 100

In [40]:
class VolatilityRiskAgent:
    """Analyzes stock price data for volatility and risk levels."""
    def __init__(self, ticker):
        self.ticker = ticker
    
    def analyze(self, price_data):
        """Computes annualized volatility and determines risk level."""
        returns = price_data["Close"].pct_change().dropna()
        volatility = returns.std() * np.sqrt(252) * 100  # Annualized volatility (%)
        risk_level = "Moderate" if 10 <= volatility < 20 else "High" if volatility >= 20 else "Low"
        dca_tip = f"DCA at ${price_data['Close'].mean():.0f} to manage {risk_level.lower()} risk"
        return {
            "volatility": round(volatility, 2),
            "risk_level": risk_level,
            "dca_tip": dca_tip
        }

In [41]:
class EducationalContentAgent:
    """Generates beginner-friendly educational content based on analysis."""
    def __init__(self):
        self.glossary = {
            "QoQ Growth": "Quarter-over-Quarter Growth: Measures revenue or profit increase from one quarter to the next.",
            "DCA": "Dollar-Cost Averaging: Investing fixed amounts regularly to reduce risk from price swings."
        }
    
    def generate_content(self, stock, report, volatility_data):
        """Generates educational content for the stock."""
        content = {
            "growth_explanation": f"{stock}'s growth is driven by {report['Growth Drivers'].lower()}.",
            "volatility_tip": volatility_data["dca_tip"],
            "glossary_term": self.glossary["QoQ Growth"] if "QoQ" in report["Growth Drivers"] else self.glossary["DCA"]
        }
        return content

In [42]:
class WatchlistHookAgent:
    """Manages a watchlist of stocks, tracks changes, and generates alerts."""
    def __init__(self):
        self.watchlist = []
    
    def add_to_watchlist(self, stock, orchestrator):
        """Adds a stock to the watchlist and generates initial metrics."""
        report = orchestrator.run_pipeline(stock)
        volatility_agent = VolatilityRiskAgent(stock)
        price_data = yf.Ticker(stock).history(period="1y")
        volatility_data = volatility_agent.analyze(price_data)
        edu_agent = EducationalContentAgent()
        edu_content = edu_agent.generate_content(stock, report, volatility_data)
        
        self.watchlist.append({
            "Stock": stock,
            "Score": report["Growth Confidence Score"],
            "Conviction Meter": min(report["Growth Confidence Score"] * 0.85, 100),  # Simplified
            "Last Alert": f"Added to watchlist with score {report['Growth Confidence Score']}"
        })
        return {
            "report": report,
            "volatility": volatility_data,
            "education": edu_content
        }
    
    def check_updates(self, stock, orchestrator):
        """Checks for changes in score or metrics and generates alerts."""
        new_report = orchestrator.run_pipeline(stock)
        old_report = next((item for item in self.watchlist if item["Stock"] == stock), None)
        if old_report and new_report["Growth Confidence Score"] != old_report["Score"]:
            alert = f"{stock} score changed to {new_report['Growth Confidence Score']}—+{new_report['Growth Confidence Score'] - old_report['Score']}"
            old_report["Last Alert"] = alert
            return alert
        return None

In [43]:
class Orchestrator:
    """Manages the pipeline for qualitative, quantitative, and additional analyses."""
    def __init__(self, ticker, api_key):
        self.ticker = ticker
        self.api_key = api_key
        self.qual_agent = QualitativeAgent(ticker, api_key)
        self.quant_agent = QuantitativeAgent(ticker)
        self.scoring_agent = ScoringAgent()
        self.vol_agent = VolatilityRiskAgent(ticker)
        self.edu_agent = EducationalContentAgent()
        self.watchlist_agent = WatchlistHookAgent()
    
    def run_pipeline(self, ticker=None):
        """Runs the full analysis pipeline for a specified ticker or the initialized ticker.
        
        Args:
            ticker (str, optional): The stock ticker to analyze. If None, uses the initialized ticker.
        """
        # Use provided ticker or fall back to initialized ticker
        target_ticker = ticker if ticker else self.ticker
        
        logging.info(f"Starting pipeline for {target_ticker}")
        
        # Update agents with the target ticker for dynamic analysis
        self.qual_agent = QualitativeAgent(target_ticker, self.api_key)
        self.quant_agent = QuantitativeAgent(target_ticker)
        self.vol_agent = VolatilityRiskAgent(target_ticker)
        
        # Qualitative analysis
        qualitative_data = self.qual_agent.get_business_model_info()
        
        # Quantitative analysis
        quantitative_data = self.quant_agent.get_financial_data()
        
        # Compute score
        final_score = self.scoring_agent.compute_score(qualitative_data, quantitative_data)
        
        # Fetch price data for volatility
        price_data = yf.Ticker(target_ticker).history(period="1y")
        volatility_data = self.vol_agent.analyze(price_data)
        
        # Generate educational content
        edu_content = self.edu_agent.generate_content(target_ticker, {
            "Growth Drivers": qualitative_data.get("growth_drivers", "N/A"),
            "Growth Confidence Score": final_score
        }, volatility_data)
        
        # Compile report
        report = {
            "Ticker": target_ticker,
            "Growth Confidence Score": final_score,
            "Business Model": qualitative_data.get("business_model", "N/A"),
            "Competitive Position": qualitative_data.get("competitive_position", "N/A"),
            "Growth Drivers": qualitative_data.get("growth_drivers", "N/A"),
            "Risks": qualitative_data.get("risks", "N/A"),
            "Narrative": qualitative_data.get("narrative", "N/A"),
            "Revenue Growth (QoQ)": quantitative_data.get("revenue_growth", "N/A") * 100,  # Convert to percentage
            "Profit Growth (QoQ)": quantitative_data.get("profit_growth", "N/A") * 100,    # Convert to percentage
            "Cash Flow Stability": quantitative_data.get("cash_flow_stability", "N/A"),
            "EPS": quantitative_data.get("EPS", "N/A"),
            "Free Cash Flow": quantitative_data.get("free_cash_flow", "N/A"),
            "Volatility": f"{volatility_data['volatility']}% ({volatility_data['risk_level']})",
            "DCA Tip": volatility_data["dca_tip"],
            "Educational Content": edu_content
        }
        
        logging.info(f"Final Score for {target_ticker}: {final_score:.2f}")
        return report

In [44]:
# Run the pipeline and demonstrate watchlist functionality
if __name__ == "__main__":
    ticker = "NVDA"
    # api_key = os.getenv("XAI_API_KEY")
    api_key = XAI_API_KEY
    if not api_key:
        raise ValueError("Grok API key is required. Set the XAI_API_KEY environment variable.")
    
    orchestrator = Orchestrator(ticker, api_key)
    report = orchestrator.run_pipeline()
    print(f"Analysis Report for {ticker}:")
    for key, value in report.items():
        print(f"{key}: {value}")
    
    # Add to watchlist and check for updates
    watchlist_result = orchestrator.watchlist_agent.add_to_watchlist(ticker, orchestrator)
    print(f"\nWatchlist Entry for {ticker}:")
    for key, value in watchlist_result["report"].items():
        print(f"{key}: {value}")
    print(f"Volatility: {watchlist_result['volatility']}")
    print(f"Educational Content: {watchlist_result['education']}")
    
    # Simulate an update check
    update_alert = orchestrator.watchlist_agent.check_updates(ticker, orchestrator)
    if update_alert:
        print(f"\nAlert: {update_alert}")

INFO:root:Starting pipeline for NVDA
INFO:httpx:HTTP Request: POST https://api.x.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Qualitative data extracted for NVDA
INFO:root:Quantitative data retrieved for NVDA
INFO:root:Final Score for NVDA: 39.93
INFO:root:Starting pipeline for NVDA


Analysis Report for NVDA:
Ticker: NVDA
Growth Confidence Score: 39.933171699737414
Business Model: TechWave Inc. operates on a subscription-based model, charging clients a monthly fee based on their usage of the company's cloud-based data analytics and machine learning solutions. This model provides recurring revenue and aligns with the growing trend of businesses seeking scalable, pay-as-you-go services.
Competitive Position: TechWave holds a strong position in the niche market of specialized data analytics and machine learning tools. While it competes with larger players like Amazon Web Services and Microsoft Azure, TechWave has differentiated itself by focusing on tailored solutions for specific industries, which has helped it carve out a loyal customer base.
Growth Drivers: The primary growth drivers for TechWave include its continuous innovation in data analytics and machine learning, which allows it to stay ahead of industry trends. Additionally, the company's ability to tailor s

INFO:httpx:HTTP Request: POST https://api.x.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Qualitative data extracted for NVDA
INFO:root:Quantitative data retrieved for NVDA
INFO:root:Final Score for NVDA: 39.93
INFO:root:Starting pipeline for NVDA



Watchlist Entry for NVDA:
Ticker: NVDA
Growth Confidence Score: 39.933171699737414
Business Model: TechWave Inc. operates on a subscription-based model, providing cloud computing services to businesses. This model ensures a steady revenue stream and scalability as customers can upgrade or downgrade their subscriptions based on their needs.
Competitive Position: TechWave holds a strong competitive position with a 25% market share in the cloud computing sector. While it competes with giants like Amazon Web Services and Microsoft Azure, TechWave differentiates itself through specialized services tailored to niche markets and superior customer support.
Growth Drivers: The key growth drivers for TechWave include the increasing global demand for cloud computing solutions, expansion into emerging markets such as Asia and Latin America, and continuous innovation in product offerings to meet evolving customer needs.
Risks: TechWave faces several risks and challenges, including intense competit

INFO:httpx:HTTP Request: POST https://api.x.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Qualitative data extracted for NVDA
INFO:root:Quantitative data retrieved for NVDA
INFO:root:Final Score for NVDA: 39.93
